### 1_basic_data_exploration.ipynb
## community data and transection data eda
目標：
✅ 載入並檢視原始資料結構
✅ 驗證PRD文件中的資料描述
✅ 識別資料品質問題

內容大綱：


In [ ]:
import os
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parent  # 找出根目錄：Path.cwd()找出現在所在目錄(/run).parent(上一層是notebook).parent(再上層一層business_district_discovery)
print(project_root)
sys.path.append(str(project_root))

In [ ]:
from utils.helper_func import convert_mixed_date_columns, calculate_presale_transaction_counts, calculate_cancellation_counts, calculate_first_transaction_dates_fast, correct_sales_start_date
from utils.helper_func import process_duplicate_communities, sample_csv_to_target_size

In [ ]:
from matplotlib.font_manager import fontManager
import matplotlib as mlp
font_path = Path(project_root) / 'utils'/"ChineseFont.ttf"
fontManager.addfont(str(font_path))
mlp.rc('font', family="ChineseFont")
print(font_path)

In [ ]:
# 預售屋備查(community data) / 實價預售交易資料(transection data )
community_dir = r"C:\pylabs\area-risk-flagging\data\lvr_moi\community_data\processed"
community_fn = "community_data.csv"
community_input_path = os.path.join(community_dir,  community_fn)


transaction_dir = r"C:\pylabs\area-risk-flagging\data\lvr_moi\transection_data\pre_sale\processed"
transaction_fn = "ps_transaction_data.csv"
transaction_input_path = os.path.join(transaction_dir,  transaction_fn)

### 2. 資料載入與基本資訊檢視

In [ ]:
print(" 載入資料檔案...")
print("=" * 80)
print(" 資料基本資訊總覽")
print("=" * 80)
try:
    # 載入預售社區資料
    community_df = pd.read_csv(community_input_path, encoding='utf-8')
    print(f" 預售社區資料載入成功: {community_df.shape}  記憶體使用: {community_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # 載入「預售」逐筆交易資料  
    transaction_df = pd.read_csv(transaction_input_path, encoding='utf-8')
    print(f" 逐筆交易資料載入成功: {transaction_df.shape}  記憶體使用: {transaction_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
except FileNotFoundError as e:
    print(f" 檔案載入失敗: {e}")
    print(" 請確認檔案是否放置在資料夾中")
except Exception as e:
    print(f" 載入過程發生錯誤: {e}")

In [ ]:
def info(df):
    """
    顯示 DataFrame 欄位資訊，包括資料型別、非空值數量、缺失數量與缺失率。
    參數：
    df (pd.DataFrame): 要檢視的資料表。
    """      
    print("-" * 50)
    summary = pd.DataFrame({
        '欄位名稱': df.columns,
        '資料型別': df.dtypes,
        '非空值數量': df.count(),
        '缺失值數量': df.isnull().sum(),
        '缺失率(%)': (df.isnull().sum() / len(df) * 100).round(2)
    }).reset_index(drop=True)

    return summary

In [ ]:
print("預售社區資料欄位資訊:")
display(info(community_df))

# 建立交易資料欄位資訊
print("逐筆交易資料欄位資訊:")
display(info(transaction_df))

In [ ]:
# 欄位型態轉換
# 面積/價格欄位
transaction_df['總面積'] = (
    transaction_df['總面積']
    .astype(str)                           # 確保是字串
    .str.replace(',', '', regex=False)    # 移除千分位逗號
    .astype(float)                        # 轉成 float
)

# 將「車位總價」欄位轉為 int64（整數），並處理可能的逗號格式
transaction_df['車位總價'] = (
    transaction_df['車位總價']
                                                   # 先轉字串
    .str.replace(',', '', regex=False)     # 移除千分位逗號
    .astype(float)                            # 先轉 float
    .round(0)                                 # 四捨五入（可選）
    .astype('Int64')                         # Pandas 支援缺值的整數型別
)

# display(community_df.dtypes, transaction_df.dtypes)

In [ ]:
# 日期欄位轉換
# 民國整數欄位 roc_integer_cols
# 民國斜線欄位（交易資料表）roc_slash_cols
# 西元欄位ad_cols

community_df = convert_mixed_date_columns(
    community_df,
    roc_cols=['銷售起始時間', '完成建物第一次登記日期', '自售起始時間', '代銷起始時間', '備查完成日期', '建照核發日'],
    ad_cols=['匯入時間']
)

transaction_df = convert_mixed_date_columns(
    transaction_df,
    roc_slash_cols=['交易日期'],
    roc_cols=['解約日期'],
    ad_cols=['匯入時間']
)

In [ ]:
# community_data 及 transaction_data轉換資料型態
# 轉換所有 object 欄位成 string
for d in [community_df, transaction_df]:
    d[d.select_dtypes(include='object').columns] = d.select_dtypes(include='object').astype('string')

In [ ]:
# 回填錯誤格式的銷售起始時間
community_df.loc[community_df['編號'] == 'G2A011008090001', '銷售起始時間'] = pd.to_datetime('2020-09-30')
community_df.loc[community_df['編號'] == 'G2G011106180002', '銷售起始時間'] = pd.to_datetime('2022-07-01')

In [ ]:
# community_df新增「預售交易筆數」欄位，計算每個預售社區的交易筆數
community_df['預售交易筆數'] = calculate_presale_transaction_counts(
    community_df, 
    transaction_df
)

In [ ]:
community_df['解約筆數'] = calculate_cancellation_counts(
    community_df, 
    transaction_df
    )

In [ ]:
# print(" 計算最初交易日期...")
community_df['最初交易日期'] = calculate_first_transaction_dates_fast(community_df, transaction_df)

In [ ]:
# 如果最初第一筆交易日期小於銷售起始日期，則以第一筆交易日期替代銷售起始日
community_df = correct_sales_start_date(community_df)

In [ ]:
# 檢查是否有重複的社區組合
community_duplicates = community_df[
    community_df.duplicated(subset=['行政區','建照執照', '經度'], keep=False)
].sort_values(by=['行政區','建照執照', '經度'])

# 顯示結果
if not community_duplicates.empty:
    print("🔁 發現重複交易紀錄如下：")
    display(community_duplicates)
else:
    print("✅ 沒有發現以『'行政區','建照執照'』為鍵的重複交易紀錄")

In [ ]:
# 重複出現的社區合併處理，並新增有效/無效欄位區別
community_df = process_duplicate_communities(community_df)

In [ ]:
# 檢視transaction_df的重複登記資料(日期+流水號)

# 檢查是否有重複的「交易日期 + 流水號」組合
duplicates = transaction_df[
    transaction_df.duplicated(subset=['交易日期', '流水號'], keep=False)
].sort_values(by=['交易日期', '流水號'])

# 顯示結果
if not duplicates.empty:
    print("🔁 發現重複交易紀錄如下：")
    print(duplicates)
else:
    print("✅ 沒有發現以『交易日期 + 流水號』為鍵的重複交易紀錄")

In [ ]:
print("預售社區資料欄位資訊:")
display(info(community_df))

# 建立交易資料欄位資訊
print("逐筆交易資料欄位資訊:")
display(info(transaction_df))

### 3. 資料樣本檢視與格式分析

In [ ]:
display(community_df.columns)
display(transaction_df.columns)

In [ ]:
community_df[community_df['預售交易筆數'] == 0].groupby(['縣市']).size()

In [ ]:
test_df = community_df[(community_df['預售交易筆數'] == 0) & (community_df['縣市'] == '臺中市')]


In [ ]:
test_df['季度'] = test_df['銷售起始時間'].dt.to_period('Q').astype(str)


In [ ]:
a = test_df.groupby(['季度'])
a.size()


In [ ]:
transaction_df[transaction_df['備查編號'] == '	G1A011008050001']

In [ ]:
# 檢視預售社區/逐筆銷售資料樣本
print("🔍 預售社區資料前5筆樣本:")
print("=" * 80)
display(community_df.head())

# %%
print("\n🔍 逐筆交易資料前5筆樣本:")
print("=" * 80)
display(transaction_df.head())

In [ ]:
# 輸出的資料夾
output_dir = r"C:\pylabs\area-risk-flagging\data\lvr_moi\community_data\processed"
os.makedirs(output_dir, exist_ok=True)

csv_fn = "community_data_proc.csv"
out_path = os.path.join(output_dir,  csv_fn)
community_df.to_csv(out_path, index=False, encoding='utf-8-sig')

pkl_fn   = "community_data_proc.pkl"
pkl_path = os.path.join(output_dir, pkl_fn)
community_df.to_pickle(pkl_path)

In [ ]:
# 輸出的資料夾
output_dir = r"C:\pylabs\area-risk-flagging\data\lvr_moi\transection_data\processed"
os.makedirs(output_dir, exist_ok=True)

csv_fn = "transection_data_proc.csv"
out_path = os.path.join(output_dir,  csv_fn)
transaction_df.to_csv(out_path, index=False, encoding='utf-8-sig')

pkl_fn   = "transection_data_proc.pkl"
pkl_path = os.path.join(output_dir, pkl_fn)
transaction_df.to_pickle(pkl_path)

In [ ]:
# 製作小型測試檔

raw_input_dir = r"C:\pylabs\area-risk-flagging\data\lvr_moi\transection_data\processed"
raw_fn = "transection_data_proc.csv"
raw_data_path = os.path.join(raw_input_dir,  raw_fn)

test_output_dir = r"C:\pylabs\area-risk-flagging\data\lvr_moi\transection_data\processed"
test_fn = "transection_data_test.csv"
test_data_path = os.path.join(test_output_dir,  test_fn)

In [ ]:
sampled = sample_csv_to_target_size(
    input_path=raw_data_path,
    output_path=test_data_path,
    target_mb=40
)

In [ ]:
def get_city_absorption_stats(community_df, city_name):
    """
    Return presale absorption statistics for a given city,
    including a summary row with total counts and net absorption rate.
    """
    # 計算統計表
    community_stats_df = (
        community_df
        .groupby(['縣市', '行政區'], as_index=False)
        .agg(
            社區數=('編號', 'nunique'),  # 統計不重複的社區編號數
            戶數總和=('戶數', 'sum'),     # 加總戶數
            銷售總和=('預售交易筆數', 'sum'),  #加總預售交易筆數
            解約總和=('解約筆數', 'sum')
            )
        .assign(
            淨去化比率=lambda df: ((df['銷售總和']-df['解約總和'])/df['戶數總和']).round(4)
            )
        .sort_values(by='淨去化比率', ascending=False)  # 按去化比率排序（由高至低）
        .reset_index(drop=True)  # 重設索引（讓排序後的 index 是連續的）
        )
    
    # Filter for the selected city
    city_df = community_stats_df[community_stats_df['縣市'] == city_name].reset_index(drop=True)

    # Create total row
    total_row = pd.DataFrame({
        '縣市': [city_name],
        '行政區': ['合計'],
        '社區數': [city_df['社區數'].sum()],
        '戶數總和': [city_df['戶數總和'].sum()],
        '銷售總和': [city_df['銷售總和'].sum()],
        '解約總和': [city_df['解約總和'].sum()]
    })

    # Compute net absorption rate for total
    total_row['淨去化比率'] = (
        (total_row['銷售總和'] - total_row['解約總和']) / total_row['戶數總和']
    ).round(4)

    # Concatenate and return
    return pd.concat([city_df, total_row], ignore_index=True)


In [ ]:
stats_df = get_city_absorption_stats(community_df, '高雄市')
display(stats_df)

In [ ]:
main_use_df = community_df['主要用途'].value_counts(dropna=False).reset_index()
main_use_df.columns = ['主要用途', '筆數']
display(main_use_df)

In [ ]:
# 檢查是否有重複的「交易日期 + 流水號」組合
community_duplicates = community_df[
    community_df.duplicated(subset=['行政區','建照執照', '經度'], keep=False)
].sort_values(by=['行政區','建照執照', '經度'])

# 顯示結果
if not community_duplicates.empty:
    print("🔁 發現重複交易紀錄如下：")
    display(community_duplicates)
else:
    print("✅ 沒有發現以『'行政區','建照執照'』為鍵的重複交易紀錄")